# An Introduction to Java 8 Features by Adam Shaver, adam.shaver@gmail.com

## Walkthrough
 * Java SE 8 was officially released March 2015.
 * Open a browser, add a link to the [Oracle Java Docs on Java SE 8](https://docs.oracle.com/javase/8/docs/api/).
 * If you're feeling brave, glance at the section on [Resources](#Resources_other_than_this_guide), and get one of the books.
 * Read the brief [Overview on Lambdas](#Overivew_Of_Lambdas). If you're drawn to the idea from JavaScript, read the [Source-Map-Sink](#lodash) section. Congragulations, you are now cooking with fire.
 * Please read the section on Lambda best [practices and pitfalls](#Pitfalls). This will make your co-workers (and future self) hate you less.
 * Understand, if you're going to invest time in crafting clear, maintainable code, please read this whole guide. Also, beyond this page, there is still quite a bit to learn.
 * This [Jupyter](http://jupyter.org/) notebook was compiled and hosted on [Cloud 9 IO](https://c9.io). It contains a non-standard [Java Kernel](https://github.com/ajlane/java8kernel) which processes Java code via a Java "Read Eval Print Loop" Jar ([REPL](https://github.com/albertlatacz/java-repl)), which is run on a copy of Java SE 8 from Oracle. That's the stack. If you want to install it, instructions are in section on [Building a Java Jupyter Notebook](#Building_a_Java_Jupyter_Notebook).  
 * Either build up this environment and learn or go to something like a publicly available [Java REPL](http://www.javarepl.com/term.html)

## Overview of Java SE 8
Java SE 8 was first made available to the general public via the Java SE 8 developer release candidate in Sept 2014. The first official release was in the following year, March 2015. When this notebook was written, the most current version available from Oracle was Java SE 8, Update 121. Release notes for that and all previous versions are available at Oracle per [version](http://www.oracle.com/technetwork/java/javase/8u-relnotes-2225394.html). As an aside, I am of the mindset that it's generally a good idea to read through the bugs fixed in updates to see if switching to a subsequent or most-recent version is necessary. 

### Timeline and Size Of Java Releases
|	Java SE	|||	Java EE	|||
|------|------|------|----|||
|Year|Version|Num Classes/Interfaces|Year|Version|Num Classes|
|1996|	0|	211|	|||
|1997|	1|	477|1998|	1|---|
|1998|	2|	1524|1999|	2|356|
|2000|	3|	1840|2001|	3|427|
|2002|	4|	2723|2003|	4|739
|2004|	5|	3279|2006|	5|1050|
|2006|	6|	3793|2009|	6|1594|
|2011|	7|	4024|2013|	7|2011|
|2014|	8|	4240|2018*|	8|---|
Source for Java SE data, [Core Java Volume I](https://www.amazon.com/Core-Java-I-Fundamentals-10th/dp/0134177304/), by Cay S. Horstman
Source for Java EE data, [Oracle Java EE Doc](http://docs.oracle.com/javaee/7/api/)

SE 8 only brings in about 216 java classes and interfaces, which is less than any other release of the language. The language modifications introduce Lambdas and, importantly, a new Date API. The JVM, Nashorn, also includes modifications such that Javascript can be run directly from Java, which postures the Java WebServer clearly against Node.js and .Net. While at first this seems swell, the JS interpreter is only implemented up to ECMAScript-262 Edition 5.1 and has no support for the [DOM or CSS](http://www.javaworld.com/article/2144908/scripting-jvm-languages/nashorn--javascript-made-great-in-java-8.html). If one needs JS, there are far better alternatives. The Java GUI development chain for desktop applications (JavaFX) was updated with several new features. 

At least once, it is worthwhile to glance over the whole set of [updates](http://www.oracle.com/technetwork/java/javase/8-whats-new-2157071.html). 

Of the full list of changes, the most relevant to the Java-JS Web Application are:
- The introduction of Lambda processing, allowing more concise data manipulation and easy parallelism
- The Date API changes, which vastly simplify time localization and conversion
- Default methods in interfaces.
- [Type annotations](https://docs.oracle.com/javase/tutorial/java/annotations/type_annotations.html) (so they can be applied anywhere that a type is used, rather than having to bake it into the type class itself).
- Unicode enhancements (to support Unicode 6.2.0)

As a first-pass introduction to the language update, I think it's important to (a) become familiar with Lambda style syntax and (b) understand how to leverage lambdas in streams. Beyond that, I would work through the other items in the above list.

## Lambda Processing (aka. Functional Processing, Point Free, Etc.)

### Overview of Lambdas <a id='Overivew_Of_Lambdas'></a>

In [1]:
:reset

All variables has been cleared


In traditional Java 7, we could declare Runnable instances, which as the name suggests can then be run. The below is an example of an implemented [Runnable](https://docs.oracle.com/javase/7/docs/api/java/lang/Runnable.html), which is itself an interface, with the `run` method overridden for custom behavior. To be precise, `r1` is an instance of a class extended anonymously from `Runnable` via override. 

In [2]:
Runnable r1 = new Runnable(){      
  @Override
  public void run(){
   System.out.println("Hello world one!");
  }
}; // note below is a regular Evaluation, not a lambda

java.lang.Runnable r1 = Evaluation$pv24fy69c5tm1ws3xh7g$1@170f6640


In [3]:
r1.run();

Hello world one!



In Java 8, functions take role closer to first-class objects (moving up from second or third class functors). Their usage still isn't as free-form as in Javascript, but it's better. In the below example, `r2` holds an anonymous function, which takes no parameters, and calls System.out.println when executed. This satisfies the functional interface of [Runnable](https://docs.oracle.com/javase/8/docs/api/java/lang/Runnable.html) in Java 8.

In [4]:
Runnable r2 = () -> System.out.println("Hello world two!"); // This produces a lambda!

java.lang.Runnable r2 = Evaluation$c09163kigonalhbf8dyu$$Lambda$213/1339159175@9a0d7ec


In [5]:
r2.run();

Hello world two!



These examples come from the Oracle Lambda Quick Start Guide, which can be found by searching their [tutorials](http://www.oracle.com/technetwork/tutorials/index.html). It's worthwhile to note that the actual code behind `Runnable` had to change from an interface to a functional interface, such that it could accept lambda functions. A short blog on that specific alteration can be found at [here](https://dzone.com/articles/introduction-functional-1).


### Lambda architecture in Java 8
The bulk of the Lamda architecture can be understood through reading two packages in the Java SE 8 Documentation that describe lambdas, followed by a package and interface that covers streams:
1. [`java.lang.FunctionalInterface`](https://docs.oracle.com/javase/8/docs/api/java/lang/FunctionalInterface.html) -- The interface that any function that takes Lambda's must satisfy.
1. [`java.util.function`](https://docs.oracle.com/javase/8/docs/api/java/util/function/package-summary.html) -- The library that contains all the Lambda specifications. The architecture of Java 8 SE Lambda's lives here.
1. [`java.util.stream`](https://docs.oracle.com/javase/8/docs/api/java/util/stream/package-summary.html) -- The library that contains streams, which is one of the most common ways that people use lambdas.
1. [`java.util.stream.Stream`](https://docs.oracle.com/javase/8/docs/api/java/util/stream/Stream.html) -- The stream interface, which includes documentation about all the things that might act on streams.

The `java.util.function` package contains several important classes, generally grouped using Oracle's syntax as:
- `Supplier` - This interface specifies how data can be generated in a way that it can be consumed by Lambdas
- `Function` - This interface specifies Lambda's themselves that transform data (from one type to another type)
- `Predicate` -This interface covers a subset of Lambda functions that take data and return type `boolean`. I still don't know why this is separate from `Function`, though I suspect it has to do with optimization of Predicates for use as filters.
- `Operator` - This interface covers a subset of Lambda functions that process data, but generate new data of the same type.
- `Consumer` - This interface specifies how Lambda processed data can be turned back into non-Lambda data.

The Oracle implementation looks extremely similar to the prior art of Guava's [Functional Ideoms](https://github.com/google/guava/wiki/FunctionalExplained). If you really want prior art and own a copy of Thinking in Java from 2006, flip to page [737-738](https://books.google.com/books?id=bQVvAQAAQBAJ&lpg=PA737&ots=LX0i9H_Y5H&dq=thinking%20in%20java%20functor&pg=PA737#v=onepage&q=thinking%20in%20java%20functor&f=false) and you'll see the initial draft of this framework a decade before it was officially codified. 

At this point, you might be thinking that the functions in Java SE 8 are not really, first-class objects. You are correct, sort of. Java really, really constrained Lambda-Stream processing to get performance gains and allow parallel processing (in what I belive was a response to popularity of Map-Reduce, Scala, and Clojure). However, we can now do really fantastic closure injection with lambdas (much like what JavaScript has been doing for the better part of a decade). It's also really important to understand the closure/context of lambdas, so we'll delve into that first.

#### First, a Trivial Closure injection, basic Lambdas
This is a trivial example showing that the closure `r3` is formed during object construction, capturing an copy of object `str`. Even when the value of the object later changes, the closure is protected. It is only when a new closure is formed `r4` that the new value of `str` is referenced. 

In [34]:
String str = "three!";

java.lang.String str = "three!"


In [35]:
Runnable r3 = () -> System.out.println("Hello world " + str);

java.lang.Runnable r3 = Evaluation$eb8ad6fo0jrv9q243msh$$Lambda$234/510763391@39e91055


In [36]:
void methodWithInjectedScope(Object r) {    
    ((Runnable) r).run();
}

Created method void methodWithInjectedScope(java.lang.Object)


In [37]:
methodWithInjectedScope(r3);

Hello world three!



Now if we change the value of `str`, it won't affect the output of our runnable, because the closure was formed during construction of the runnable.

In [38]:
str = "four!";
methodWithInjectedScope(r3); // note closure does not change.

Hello world three!
java.lang.String str = "four!"


However, if we form a new runnable, referencing the new value of `str` in the closure, we will see the changed value.

In [39]:
Runnable r4 = () -> System.out.println("Hello world " + str);

java.lang.Runnable r4 = Evaluation$qh0ojrcfu8xylvpt5a3e$$Lambda$235/957260889@6fde09e4


In [40]:
methodWithInjectedScope(r4); // note closure references the new value of 'str'. 

Hello world four!



We can even implement a more complicated version, where the `str` value is wrapped inside another object. This lets us change the closure, by maintaining a reference to the referenced object.

In [12]:
:reset

All variables has been cleared


In [41]:
class WrappedStrExample {
    class WrappedStr {
        public String str = "Three.";
    }
    
    private WrappedStr wstr = new WrappedStr();
    private String str = "Three.";
    
    public Runnable r4 = null;
    public Runnable r3 = null;

    WrappedStrExample() {
        r3 = () -> System.out.println("Hello world " + this.str);
        r4 = () -> System.out.println("Hello world Wrapped " + this.wstr.str);
    };

    public void SetStr(String str) {
        this.str = str;
    };

    public void SetWStr(String str) {    
        this.wstr.str = str;
    };
}

Created type WrappedStrExample


In [42]:
WrappedStrExample wse = new WrappedStrExample();

java.lang.Object wse = WrappedStrExample@129e258b


In [43]:
((WrappedStrExample) wse).r3.run();
((WrappedStrExample) wse).r4.run();

Hello world Three.
Hello world Wrapped Three.



In [44]:
((WrappedStrExample) wse).SetStr("Five"); // Alters the closure referenced by the runnable 

In [45]:
((WrappedStrExample) wse).SetWStr("Five"); // Also  alters the closure referenced by the runnable

In [46]:
((WrappedStrExample) wse).r3.run();
((WrappedStrExample) wse).r4.run();

Hello world Five
Hello world Wrapped Five



#### Second, Non-Trivial Object Construction with Closure injection (think object localization)
This example shows how closure and lambdas can be used together to make flexible code. This exploits the closure to make two different constructors that each have different values of scoped variables in their closure. The pattern lends itself to flexible construction, handling dates, polymorphic behaviors for control objects, etc.

In [48]:
class CurrencyType {
    public String name = ""; 
    public String country = "";
    public CurrencyType(String name,  String country){
        this.name = name;
        this.country = country;
    }
};

Created type CurrencyType


In [49]:
class Money {
    public CurrencyType type;  
    public int quantity=0; 
    public String container;
    public String toString() {
        return ("Name: " + type.name + ", Country: " + type.country + ", Quantity: " + this.quantity + ", Container: " + this.container);
    };
};

Created type Money


We can build a factory from a lambda inside our the class with strong typing in the Lambda.

In [51]:
Money makeWallet(CurrencyType type) {
    
    Function<CurrencyType, Money> wallet = (CurrencyType c) -> {
        Money money = new Money(); 
        money.type =  c;
        money.quantity = 100;
        money.container = "Wallet";
        return money;
    };
    return wallet.apply(type);
};

Created method java.lang.Object makeWallet(CurrencyType)


We can also build and inject a factory from outside scope, though we have to cast (because the compile has trouble figuring out types with the injection of the anonymous lambda. My suspicion is that this is due to type erasure at runtime).

In [52]:
Function<CurrencyType, Money> atm = ( c) -> {
    Money money = new Money(); 
    money.type = (CurrencyType) c;
    money.quantity = 100000;
    money.container = "ATM";
    return money;
};

java.util.function.Function<java.lang.Object, java.lang.Object> atm = Evaluation$7rgj9cx5hwtdpbmlfoqi$$Lambda$238/461691259@10992ee4


In [53]:
Money makeAtm(CurrencyType type) {    
    return (Money) atm.apply(type); 
}; 

Created method java.lang.Object makeAtm(CurrencyType)


Now we can simply construct instances of different types, by using lambdas in a factory pattern.

In [22]:
makeWallet(new CurrencyType("Dollar", "Canada"));

java.lang.Object res2 = Name: Dollar, Country: Canada, Quantity: 100, Container: Wallet


In [54]:
makeAtm(new CurrencyType("Dollar", "USA"));

java.lang.Object res7 = Name: Dollar, Country: USA, Quantity: 100000, Container: ATM


#### Third, Lambda's and streams, the Source-Map-Sink pattern (or how to replace the Java `for` loop and pretend you're using underscore or lodash from JavaScript)<a id='lodash'></a>
Before we go any further, I think it's useful to split the contents of java.util.stream into what I call source, processing, and sink types. Sources generate data that is processed and may eventually be collected in a terminal sink. I know that is a fairly loose classification, but it's my feel of how the interfaces are patterned. Basically, you start by turning a collection into a stream, then use lambdas to do processing, then turn it back into a collection via one of the [`Collectors`](https://docs.oracle.com/javase/8/docs/api/java/util/stream/Collectors.html). That being said, it's also common to use a constructor or builder to satisfy the [`Stream.Builder`](https://docs.oracle.com/javase/8/docs/api/java/util/stream/Stream.Builder.html) interface. 

The key points in this process are (a) generating the stream, (b) deciding which functional interface to use, (c) converting the stream back to a collection, and (d) checking everything throughly for bugs. 

##### Step 1, generate a stream
 - from a collection using `.stream()` or `.parallelStream()`
 - from a stream builder using a constructor/factory/builder to generate the stream, item by item
 - from any other Supplier (even you you make) that can can produce a stream according to the stream interface.

##### Step 2, act on the objects in the stream. Beware these are lazy, so if there is no terminal action, they won't be executed.
 - filter, map, flatmap, distinct, sorted, peek, limit, skip, etc.
 - basically applying those Functions, Predicates, and Operators
 
##### Step 3, terminal operations
 - forEach, min, max, count, findFirst, findAny, anyMatch, collect, reduce, toArray, etc.
 - applying the Consumers

Below is an example. Look at methods **`topUp()`** and **`simulateDay()`** for lambdas and **`stream()`** and **`parallelStream()`** to see how easy it is to do parallelization.

In [26]:
:reset

All variables has been cleared


In [1]:
class Bank {
    
    /* Defines Currency specifics*/
    class CurrencyType {
        public String name = ""; 
        public String country = "";
        public CurrencyType(String name,  String country){
            this.name = name;
            this.country = country;
        }
    };
    
    /* Defines the container that holds currency (e.g., bank, atm, wallet, credit card)*/
    class MoneyContainer {
        public CurrencyType type;  
        public int quantity=0; 
        public String storageMethod;
        public String toString() {
            return ("Name: " + type.name + ", Country: " + type.country + ", Quantity: " + this.quantity + ", Storage: " + this.storageMethod);
        };
    };
    
    /* Generator that makes an ATM of a currency type*/
    Function<CurrencyType, MoneyContainer> atm = (type) -> {
        MoneyContainer mc = new MoneyContainer(); 
        mc.type = (CurrencyType) type;
        mc.quantity = 100000;
        mc.storageMethod = "ATM";
        return mc;
    };
    
    /* Factory to make ATMs with a lambda*/
    MoneyContainer makeMoneyAtm(CurrencyType type) {    
        return (MoneyContainer) atm.apply(type); 
    }; 
    
    List<MoneyContainer> moneyContainerList = new ArrayList<>();
    
    /* Set up a bank of 5 atms*/
    Bank() {
        for (int i = 0; i < 5; i++) {
            moneyContainerList.add(makeMoneyAtm(new CurrencyType("Dollar", "USA")));
        }        
    };
    
    /* REplace all the atms with full containers 10000 */
    public List<MoneyContainer> topUp() {
        moneyContainerList = moneyContainerList.stream()
            .map(mc -> {return makeMoneyAtm(new CurrencyType("Dollar", "USA"));})
            .collect(java.util.stream.Collectors.toList());
        return moneyContainerList;
    };
    
    /* Simulate withdrawl over a day. Also, please note improper use of filter (so we can show below hash examples).*/
    public List<MoneyContainer> simulateDay() {
        moneyContainerList = moneyContainerList.parallelStream()
            .filter(mc -> {mc.quantity -= (Math.random() * 50000); return true;} )
            .collect(java.util.stream.Collectors.toList());
        return moneyContainerList;
    };
    
    /* Simulate withdrawl over a day. Note, there is no return from the collector, stream objects are mutated.*/
     public void simulateDayNoReturn() {
        moneyContainerList.parallelStream()
            .filter(mc -> {mc.quantity -= (Math.random() * 50000); return true;} )
            .collect(java.util.stream.Collectors.toList());
    };
    
     /* Simulate withdrawl over a day. Note, there is no terminal operation (e.g., filter is non-terminal)*/
     public List<MoneyContainer> simulateDayNoTerminal() {
        moneyContainerList.parallelStream()
            .filter(mc -> {mc.quantity -= (Math.random() * 50000); return true;} );
         return moneyContainerList;
    };
    
    public void printHoldings() {
        moneyContainerList.stream()
            .forEach(mc -> {System.out.println(mc);} );
    }
    
    public void printHashCode() {
        for (MoneyContainer mc : moneyContainerList) {
            System.out.println(mc.hashCode());
        }
    }
    
    public void printHashCodeInStream() {
        moneyContainerList.stream()
            .forEach(mc -> {System.out.println(mc.hashCode());} );
    }
    
    public void printHashCodeInStream2() {
        moneyContainerList.stream().filter(mc -> {return true;})
            .peek(mc -> {System.out.println(mc.hashCode());} )
            .collect(java.util.stream.Collectors.toList());
    }
}

Created type Bank


In [2]:
Bank abank = new Bank();

java.lang.Object abank = Bank@7e7205ae


In [3]:
((Bank) abank).printHoldings();

Name: Dollar, Country: USA, Quantity: 100000, Storage: ATM
Name: Dollar, Country: USA, Quantity: 100000, Storage: ATM
Name: Dollar, Country: USA, Quantity: 100000, Storage: ATM
Name: Dollar, Country: USA, Quantity: 100000, Storage: ATM
Name: Dollar, Country: USA, Quantity: 100000, Storage: ATM



In [4]:
((Bank) abank).simulateDay();
((Bank) abank).printHoldings();

Name: Dollar, Country: USA, Quantity: 62511, Storage: ATM
Name: Dollar, Country: USA, Quantity: 86202, Storage: ATM
Name: Dollar, Country: USA, Quantity: 70263, Storage: ATM
Name: Dollar, Country: USA, Quantity: 95266, Storage: ATM
Name: Dollar, Country: USA, Quantity: 52017, Storage: ATM
java.util.ArrayList res1 = [Name: Dollar, Country: USA, Quantity: 62511, Storage: ATM, Name: Dollar, Country: USA, Quantity: 86202, Storage: ATM, Name: Dollar, Country: USA, Quantity: 70263, Storage: ATM, Name: Dollar, Country: USA, Quantity: 95266, Storage: ATM, Name: Dollar, Country: USA, Quantity: 52017, Storage: ATM]


In [5]:
System.out.println("Refill");
((Bank) abank).topUp();
((Bank) abank).printHoldings();

Refill
Name: Dollar, Country: USA, Quantity: 100000, Storage: ATM
Name: Dollar, Country: USA, Quantity: 100000, Storage: ATM
Name: Dollar, Country: USA, Quantity: 100000, Storage: ATM
Name: Dollar, Country: USA, Quantity: 100000, Storage: ATM
Name: Dollar, Country: USA, Quantity: 100000, Storage: ATM



In [58]:
((Bank) abank).printHashCode();

774312511
1121580136
379786833
1683659815
1929038969



In [59]:
((Bank) abank).printHashCodeInStream();

774312511
1121580136
379786833
1683659815
1929038969



In [60]:
((Bank) abank).printHashCodeInStream2();

774312511
1121580136
379786833
1683659815
1929038969



Note, given that the above hash codes are the same, does one need to actually have a collector in the stream at all?

In [61]:
((Bank) abank).printHoldings();
((Bank) abank).simulateDayNoReturn();
((Bank) abank).printHoldings();

Name: Dollar, Country: USA, Quantity: 100000, Storage: ATM
Name: Dollar, Country: USA, Quantity: 100000, Storage: ATM
Name: Dollar, Country: USA, Quantity: 100000, Storage: ATM
Name: Dollar, Country: USA, Quantity: 100000, Storage: ATM
Name: Dollar, Country: USA, Quantity: 100000, Storage: ATM
Name: Dollar, Country: USA, Quantity: 57649, Storage: ATM
Name: Dollar, Country: USA, Quantity: 65594, Storage: ATM
Name: Dollar, Country: USA, Quantity: 99635, Storage: ATM
Name: Dollar, Country: USA, Quantity: 72507, Storage: ATM
Name: Dollar, Country: USA, Quantity: 71418, Storage: ATM



Not really, but the stream still needs to have something in place as a terminal operation, otherwise it will not execute. See `simulateDayNoTerminal()` for a method that really doesn't do anything.

In [4]:
((Bank) abank).topUp();
((Bank) abank).printHoldings();

Name: Dollar, Country: USA, Quantity: 100000, Storage: ATM
Name: Dollar, Country: USA, Quantity: 100000, Storage: ATM
Name: Dollar, Country: USA, Quantity: 100000, Storage: ATM
Name: Dollar, Country: USA, Quantity: 100000, Storage: ATM
Name: Dollar, Country: USA, Quantity: 100000, Storage: ATM
java.util.ArrayList res2 = [Name: Dollar, Country: USA, Quantity: 100000, Storage: ATM, Name: Dollar, Country: USA, Quantity: 100000, Storage: ATM, Name: Dollar, Country: USA, Quantity: 100000, Storage: ATM, Name: Dollar, Country: USA, Quantity: 100000, Storage: ATM, Name: Dollar, Country: USA, Quantity: 100000, Storage: ATM]


In [3]:
((Bank) abank).simulateDayNoTerminal();

java.util.ArrayList res1 = [Name: Dollar, Country: USA, Quantity: 100000, Storage: ATM, Name: Dollar, Country: USA, Quantity: 100000, Storage: ATM, Name: Dollar, Country: USA, Quantity: 100000, Storage: ATM, Name: Dollar, Country: USA, Quantity: 100000, Storage: ATM, Name: Dollar, Country: USA, Quantity: 100000, Storage: ATM]


In [4]:
((Bank) abank).printHoldings();

Name: Dollar, Country: USA, Quantity: 100000, Storage: ATM
Name: Dollar, Country: USA, Quantity: 100000, Storage: ATM
Name: Dollar, Country: USA, Quantity: 100000, Storage: ATM
Name: Dollar, Country: USA, Quantity: 100000, Storage: ATM
Name: Dollar, Country: USA, Quantity: 100000, Storage: ATM



### Lambda Guidelines and Pitfalls<a id='Pitfalls'></a>

#### 1. Debugging lambdas/streams/anonymous functions can be difficult
Lambdas and anonymous functions may be injected (or created) at runtime, leading to issues with binding source to bytecode. If you have trouble getting breakpoints to stick, the best option is to put a breakpoint in one of the calling functions, and then to step into the method of interest. As for streams, sometimes `peek()` can be of use.
##### 2. Lambdas encourage poor variable naming
Because of the intentional lack of formalized type signatures in lambdas (e.g., n -> n+1), some developers will devolve the variable names into simple letters. To quote one of my collegues when asked if variable names matter, [Jason Wohlgemuth](https://github.com/jhwohlgemuth), "Naming is one of the major things that developers do." I am of the mindset that naming variables sensibly is still imporantant, even in lambdas. There are exeptions to everything, but I think it's best to pretend when writing a lambda, that the naming practices from non-lambda coding should still apply.
##### 3. Context/closure can do surprising things
When a lambda takes a closure, and that closure is mutated, the lambda can be affected. So, be careful about how outside scoped variables are used and referenced within a lambda (see the simple string reference above).
##### 4. Erasure can cause lots of issues
Just look at the above examples. Erasure of variable types can require lots of casting. This can be prevented by construction and usage in the same scope (class or method), such that the compiler can figure out types easily. Otherwise, typing has to be used.
##### 5. Lazy evaluation
Understand that Streams are evaluated in a lazy manner. If there is no consumer, the code is effectively dead and should probably be removed.

## Resources other than this guide <a id='Resources_other_than_this_guide'></a>
 There are several official Oracle resources for the Java 8 update:
 - Make a bookmark to the [official Java 8 SE](https://docs.oracle.com/javase/8/docs/api/) language spec provided by Oracle's java-doc processing.
 - Go through one of the official online tutorials on Lambdas such as [Oracle's Lambda Quick Start](http://www.oracle.com/webfolder/technetwork/tutorials/obe/java/Lambda-QuickStart/index.html).
 - For the date api changes, there are other tutorials such as [Oracles's Java 8 Date and Time Offset](http://www.oracle.com/webfolder/technetwork/tutorials/obe/java/dateTimeAPI/DateTimeAPI.html)
Note, the correct way to search and access Oracle tutorials is via their search engine [link](http://apex.oracle.com/pls/apex/f?p=44785:1).

I spent alot of time searching for any books that I considered worthwhile. From the Java 8 developer release candidate (Sept 2014) to the official release (March 2015) to the current date of writing this (March 2017), there are loads of books. There are several early money-grab books that just document the code and API, things that you can and probably should reference from Oracle's website. In constrast, the latter books provide better guidance, with established usage patterns. Books that I own on the subject are below in recommended in order. The most cherished are the Horstmann series, which read like novels. As an aside, the first `Fundamentals` volume has, hands down, the best section I've ever read on annotations:

-  [Core Java for the Impatient 1st Ed., Cay S. Horstmann, 2015](https://www.amazon.com/Core-Java-Impatient-Cay-Horstmann/dp/0321996321/) -- A great quick ~ 500 page overview of the language with the Java SE 8 update.
-  [Core Java I Fundamentals, 10th Ed, Cay S. Horstman, 2016](https://www.amazon.com/Core-Java-I-Fundamentals-10th/dp/0134177304/) -- A good comprehensive (nigh monolithic) overview of Java SE up to and including SE 8. It has amazing tables that document the class changes per language version, such as the table on java.lang.String on page 72. These are invaluable if you want to pick up where you last left off. (To see an example, search the contents of his book on Amazon with the phrase "We also list the version number in which a particular class was introduced".) This contains the bulk of the writing on lambdas.
-  [Core Java II Advanced Features, 10th Ed, Cay S. Horstman, 2017](https://www.amazon.com/Core-Java-II-Advanced-Features-10th/dp/0134177290/) -- As second comprehensive (also monolithic) book on Java up through SE 8. This has some useful and rarely documented information about things like user authentication and security, but also large sections that I find kind of irrelevant for what I work on (e.g., AWT, aka. Java's GUI language). 

-  [Java 8 in Action: Lambdas, Streams, and functional-style programming 1st, Urma, Fusco, Mycroft, 2014](https://www.amazon.com/Java-Action-Lambdas-functional-style-programming/dp/1617291994/) -- This book was the best I had found until I ran into the Horstman series. It is fairly standard and covers Lambdas in good depth. It would stand in easily for the content of Oracle SE 8 training on the subject. However, at times, it feels like it's just covering the same canned dialogue as every other book published in the JAVA SE 8 land-grab. To be fair, the lead author works for Oracle and it was written in August 2014 before the Developer Candidate release, so it is entirely possible every other book heavily borrowed content.

-  [Java: The Complete Reference, Ninth Edition 9th Edition, Herbert Schildt, 2014](https://www.amazon.com/gp/product/0071808558/ref=oh_aui_search_detailpage?ie=UTF8&psc=1) -- This book is mostly a dead-tree reference collection of java-docs (which are free online). It is useful to me only when I cannot access the internet (or when I want to learn offline). For that, it's perfect. The rest of the time, it holds down my desk.


## Building a Java Jupyter Notebook<a id='Building_a_Java_Jupyter_Notebook'></a>
Follow these steps:
1. Fire up a Linux box. To date, this doesn't work on Windows (due to path issues python java8kernel)
1. Get a recent copy of Jupyter from Anaconda python. I use the miniconda2 for space
    1. `wget https://repo.continuum.io/miniconda/Miniconda2-latest-Linux-x86_64.sh`
    2. `bash ./Miniconda2-latest-Linux-x86_64.sh` 
    3. `conda install jupyter'
1. Get the REPL Jupyter notebook branch from github [ajlane/java8kernel] (https://github.com/ajlane/java8kernel)
    1. clone the repo
    1. read the install instructions to install the kernel
    1. grab the compiled [javarepl.jar](http://albertlatacz.published.s3.amazonaws.com/javarepl/javarepl.jar) if you want to avoid compiling with gradel

OR,
1. Copy the cloud9 instance I used to make run and build this notebook, which is publicly available read-only via https://ide.c9.io/ashaver/java